In [3]:
import cv2
import mediapipe as mp
import os
import math

In [4]:
# Setup
videos_folder = "./input/test"
old_ann_folder = "./input/fall/old_annotation"
new_ann_folder = "./input/fall/new_annotation"
os.makedirs(new_ann_folder, exist_ok=True)

In [5]:
# MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)

In [6]:
def get_coords(landmarks, idx):
    pt = landmarks[idx]
    return (pt.x, pt.y) if pt.visibility > 0.5 else (None, None)

def is_valid(*coords):
    for c in coords:
        if c is None or c[0] is None or c[1] is None:
            return False
    return True

def extract_pose_line(lm):
    l_sh, r_sh = get_coords(lm, 11), get_coords(lm, 12)
    l_hip, r_hip = get_coords(lm, 23), get_coords(lm, 24)
    l_knee, r_knee = get_coords(lm, 25), get_coords(lm, 26)
    l_ankle, r_ankle = get_coords(lm, 27), get_coords(lm, 28)

    if not is_valid(l_sh, r_sh, l_hip, r_hip, l_knee, r_knee, l_ankle, r_ankle):
        return None

    mid_sh = [(l_sh[0] + r_sh[0]) / 2, (l_sh[1] + r_sh[1]) / 2]
    mid_hip = [(l_hip[0] + r_hip[0]) / 2, (l_hip[1] + r_hip[1]) / 2]

    line = mid_sh + mid_hip + list(l_hip) + list(r_hip) + list(l_hip) + list(l_knee) + list(l_ankle) + list(r_hip) + list(r_knee) + list(r_ankle)
    return line

In [7]:
def update_annotation_file(video_path, old_txt_path, new_txt_path):
    cap = cv2.VideoCapture(video_path)
    frame_id = 0

    with open(old_txt_path, "r") as f:
        old_lines = f.readlines()
        header_1 = old_lines[0].strip() if len(old_lines) > 0 else "0"
        header_2 = old_lines[1].strip() if len(old_lines) > 1 else "0"

    with open(new_txt_path, "w") as out_file:
        out_file.write(f"{header_1}\n{header_2}\n")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = pose.process(frame_rgb)

            if result.pose_landmarks:
                lm = result.pose_landmarks.landmark
                pose_line = extract_pose_line(lm)
                if pose_line:
                    out_file.write(f"{frame_id} " + " ".join([f"{v:.6f}" for v in pose_line]) + "\n")
                else:
                    out_file.write(f"{frame_id} " + " ".join(["-1"] * 20) + "\n")
            else:
                out_file.write(f"{frame_id} " + " ".join(["-1"] * 20) + "\n")

            frame_id += 1

    cap.release()
    print(f"✅ Saved updated annotation: {os.path.basename(new_txt_path)}")

In [8]:
for file in os.listdir(videos_folder):
    if file.endswith(".avi"):
        base = os.path.splitext(file)[0]
        video_path = os.path.join(videos_folder, file)
        old_txt_path = os.path.join(old_ann_folder, f"{base}.txt")
        new_txt_path = os.path.join(new_ann_folder, f"{base}.txt")

        if os.path.exists(old_txt_path):
            update_annotation_file(video_path, old_txt_path, new_txt_path)
        else:
            print(f"⚠️ Skipped {file} — no matching annotation found.")

pose.close()

✅ Saved updated annotation: video (1).txt
✅ Saved updated annotation: video (10).txt
✅ Saved updated annotation: video (11).txt
✅ Saved updated annotation: video (12).txt
✅ Saved updated annotation: video (13).txt
✅ Saved updated annotation: video (14).txt
✅ Saved updated annotation: video (15).txt
✅ Saved updated annotation: video (16).txt
✅ Saved updated annotation: video (17).txt
✅ Saved updated annotation: video (18).txt
✅ Saved updated annotation: video (19).txt
✅ Saved updated annotation: video (2).txt
✅ Saved updated annotation: video (20).txt
✅ Saved updated annotation: video (21).txt
✅ Saved updated annotation: video (22).txt
✅ Saved updated annotation: video (23).txt
✅ Saved updated annotation: video (24).txt
✅ Saved updated annotation: video (25).txt
✅ Saved updated annotation: video (26).txt
✅ Saved updated annotation: video (27).txt
✅ Saved updated annotation: video (28).txt
✅ Saved updated annotation: video (29).txt
✅ Saved updated annotation: video (3).txt
✅ Saved update